In [3]:
from hnefatal.game import *
from tests.test_game import run_all_tests

run_all_tests()

. . . . O O O O O . . . .
. . . . . . O . . . . . .
. . . . . . . . . . . . .
. . . . . . . . . . . . .
O . . . . . X . . . . . O
O . . . . X X X . . . . O
O O . . X X K X X . . O O
O . . . . X X X . . . . O
O . . . . . X . . . . . O
. . . . . . . . . . . . .
. . . . . . . . . . . . .
. . . . . . O . . . . . .
. . . . O O O O O . . . .

### All tests passed! ###


In [ ]:
import torch
from torch import nn
from sklearn.preprocessing import OneHotEncoder
import numpy as np

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

BOARD_WIDTH = 13

# The Neural Network is designed with a OneHot-Encoded input where each piece type is represented as a separate feature.
# The output is a tensor of shape (BOARD_WIDTH, BOARD_WIDTH, 4, BOARD_WIDTH - 1) representing the policy for each piece's movement in four directions (up, down, left, right) and distances from 1 to BOARD_WIDTH - 1.

class SimpleDefendersModel(nn.Module):
    def __init__(self):
        super(SimpleDefendersModel, self).__init__()
        self.flatten = nn.Flatten()
        self.nn = nn.Sequential(
            nn.Linear(BOARD_WIDTH**2*4, 1024),
            nn.ReLU(),
            # nn.Linear(512, 512),
            # nn.ReLU(),
            nn.Linear(1024, BOARD_WIDTH**2 * 4 * BOARD_WIDTH - 1),  # Output for each direction and a length of maximum BOARD_WIDTH - 1
            nn.Softmax(dim=-1)  # Softmax to get probabilities for each move
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.nn(x)
        return logits
    
model = SimpleDefendersModel().to(device)
print(model)

encoder = OneHotEncoder()
encoder.fit([[Piece.EMPTY.value], [Piece.ATTACKER.value], [Piece.DEFENDER.value], [Piece.KING.value]])

def apply_direction(row, col, direction, distance):
    if direction == 0:   # Up
        return row - distance, col
    elif direction == 1: # Down
        return row + distance, col
    elif direction == 2: # Left
        return row, col - distance
    elif direction == 3: # Right
        return row, col + distance

def find_ai_move(game, player):    
    # Flatten the board and get the value of each piece
    flat_board = [piece.value for row in game.board for piece in row]
    flat_board = torch.tensor(flat_board).reshape(-1, 1)
    x = encoder.transform(flat_board)
    x = torch.tensor(x.toarray(), dtype=torch.float32).to(device)

    # Get the model's prediction
    logits = model(x.flatten().unsqueeze(0))

    flat_policy = logits.reshape((BOARD_WIDTH, BOARD_WIDTH, 4, BOARD_WIDTH - 1))
    
    moves = []
    for row in range(BOARD_WIDTH):
        for col in range(BOARD_WIDTH):
            for direction in range(4):
                for distance in range(BOARD_WIDTH - 1):
                    prob = flat_policy[row, col, direction, distance]
                    # if prob > 0:
                    moves.append(((row, col, direction, distance + 1), prob))

    moves.sort(key=lambda x: x[1], reverse=True)

    # Filter moves to only valid ones
    valid_moves = []
    for move, prob in moves:
        from_row, from_col, direction, steps = move
        to_row, to_col = apply_direction(from_row, from_col, direction, steps)
        if not all(0 <= pos < BOARD_WIDTH for pos in (from_row, from_col, to_row, to_col)):
            continue

        from_pos = Coord(from_row, from_col)
        to_pos = Coord(to_row, to_col)

        if game.is_valid_move(from_pos, to_pos):
            return Move(from_pos, to_pos)

    assert False, "No valid moves found"

def test_ai_move():
    game = Game()
    game.fill_board_13_by_13()
    move = find_ai_move(game, Player.DEFENDER)

test_ai_move()

Using cpu device
SimpleDefendersModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (nn): Sequential(
    (0): Linear(in_features=676, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=8112, bias=True)
  )
)


In [5]:
import random

def find_random_move(game: Game, player: Player):
    valid_from_positions = [Coord(x, y) for x, row in enumerate(game.board) for y, piece in enumerate(row) if piece in player]
    assert len(valid_from_positions) > 0, "No valid from positions found"

    while True:
        from_pos = random.choice(valid_from_positions)

        valid_to_positions = [Coord(x, y) for x in range(13) for y in range(13) if game.is_valid_move(from_pos, Coord(x, y))]
        if len(valid_to_positions) < 1:
            continue  # No valid moves from this position, try another from_pos

        to_pos = random.choice(valid_to_positions)

        return Move(from_pos, to_pos)

def find_human_move(game: Game, player: Player):
    while True:
        try:
            from_x = int(input(f"Player {player}, enter the x-coordinate of the piece to move (0-12): "))
            from_y = int(input(f"Player {player}, enter the y-coordinate of the piece to move (0-12): "))
            to_x = int(input(f"Player {player}, enter the x-coordinate to move to (0-12): "))
            to_y = int(input(f"Player {player}, enter the y-coordinate to move to (0-12): "))

            from_pos = Coord(from_x, from_y)
            to_pos = Coord(to_x, to_y)

            if game.is_valid_move(from_pos, to_pos):
                return Move(from_pos, to_pos)
            else:
                print("Invalid move. Please try again.")
        except ValueError:
            print("Invalid input. Please enter integers between 0 and 12.")

In [6]:
import time

def single_game():
    game = Game()
    game.fill_board_13_by_13()

    player = Player.ATTACKER  # Attacker starts the game
    # print(f"Starting game with player: {player}")

    moves = 0
    game_time = time.time()
    thinking_times = []
    while not game.is_game_over():

        if player == Player.DEFENDER:
            start = time.time()
            move = find_ai_move(game, player)
            moves += 1
            thinking_times.append(time.time() - start)
        else:
            move = find_random_move(game, player)

            defender_count = sum(piece == Piece.DEFENDER for row in game.board for piece in row)
            if defender_count < 1:
                print("Game Over: No defenders left")
                break

        game.move_piece_and_attack(move.from_pos, move.to_pos)
        
        # Switch players
        player = Player.ATTACKER if player == Player.DEFENDER else Player.DEFENDER

        if moves % 100 == 0:
            print(f"Moves: {moves}, Time: {time.time() - game_time:.2f} seconds")
            print(f"Average thinking time for AI: {sum(thinking_times) / len(thinking_times):.2f} seconds" if len(thinking_times) > 0 else "")
            game.print_board()
    game.print_board()
    attacker_count = sum(piece == Piece.ATTACKER for row in game.board for piece in row)
    defender_count = sum(piece == Piece.DEFENDER for row in game.board for piece in row)
    print(f"Attackers left: {attacker_count}, Defenders left: {defender_count}")
    print(f"Game Over after {moves} moves, in {int(time.time() - game_time)} seconds")

single_game()

Moves: 0, Time: 0.00 seconds

. . . . O O O O O . . . .
. . . . . . O . . . . . .
. . . . . . . . . . . . .
. . . . . . . . . . . . .
O . . . . . X . . . . . O
O . . . . X X X . . . . O
O O . . X X K X X . . O O
O . . . . X X X . . . . O
O . . . . . X O . . . . .
. . . . . . . . . . . . .
. . . . . . . . . . . . .
. . . . . . O . . . . . .
. . . . O O O O O . . . .



KeyboardInterrupt: 